# Задача 1


В первой задаче необходимо оценить вероятность наличия в объявлении контактной информации. 
Результатом работы модели является `pd.DataFrame` с колонками:
* `index`: `int`, положение записи в файле;
* `prediction`: `float` от 0 до 1.

# Решение
## Импорт данных

- В этом задании вам предстоит реализовать решение для поиска контактов в объявлениях
- Таргет `is_bad`. Если `is_bad = 1` - контакты в объявлении есть, иначе нет
- В `train.csv` разметка может быть неточная
- Задача показать результат **ROC AUC >= 0.8**
- ROC AUC будет **усредняться по каждой категории**: оценка сначала в разрезе каждой категории, а потом усреднение
- **ВАЖНО:** На платформе установлены следующие библиотеки (необходимо ограничиться ими):

```
joblib==1.3.2
numpy==1.23.5
pandas==2.2.2
scipy==1.11.4
scikit-learn==1.4.2
lightgbm==4.3.0'
```

Поля в тренировочном `train.csv` и проверочном датасетах:
* `title` - заголовок,
* `description` - описание,
* `subcategory` - подкатегория,
* `category` - категория,
* `price` - цена,
* `region` - регион,
* `city` - город,
* `datetime_submitted` - дата размещения

In [ ]:
# ваш импорт

In [ ]:
train = pd.read_csv(....)

## Преобразование признаков

**Код с кастомными классами, а также со всеми необходимыми библиотеками, необходимо сохранить в отдельном файле с разширением *.py!!**

In [ ]:
# Здесь должен быть ваш кастомный класс для дополнительной обработки данных
# например, хотите использовать регулярные выражения. В результате обработк при помощи такого
# класса получаете ЧИСЛЕННОЕ значение

from scipy.sparse import csr_matrix
from sklearn.base import BaseEstimator, TransformerMixin


CONSTANT = # константны, можно сюда писать паттерны - опционально

class YourClassNumber(BaseEstimator, TransformerMixin):
    def __init__(self, pattern ** kwargs):
        self.pattern = pattern
        super().__init__(**kwargs)

    def fit(self, X, y=None):
        return self

    def preprocessing(self, data: str):
        ###
        return #

    def transform(self, X, y=None):
        result = csr_matrix(X.apply(self.preprocessing)).T
        return result

    def get_feature_names(self):  # опционально
        return #

In [ ]:
# Здесь должен быть ваш кастомный класс для дополнительной обработки данных
# например, хотите обработать текст и удалить лишние символы. В результате обработк при помощи такого
# класса получаете ТЕКСТОВЫЕ признаки


class YourClassText(BaseEstimator, TransformerMixin):

    def preprocessing(self, s):
        #
        return  #

    def to_desc(self, s):
        return self.preprocessing(s)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.apply(self.to_desc)

## Тренировка модели

Для простоты построим pipline, чтобы далее можно было удобно применять на тестовых данных

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer


SPLIT = 0.2
RANDOM_STATE = 10

# Создаем полный pipeline
full_pipeline = Pipeline([
    # процесс обработки фичей
    ('preprocessing', ColumnTransformer(
        [
            # обработка числовых значений - опционально
            ('numerical', Pipeline([
                ('imputer', SimpleImputer(strategy=....)),
                ('scaler', StandardScaler())]), ['признак']), 
            # обработка категориальных значений - опционально
            ('categorical', OneHotEncoder(handle_unknown='ignore', drop=...), ['признак1', 'признак2', ...]),
            # обработка тестовых признаков -> получаем числа - опционально
            ('description_regex', YourClassNumber(pattern= CONSTANT ...), 'description'),
            # обработка тестовых признаков -> сначала получаем строки, потом переводим в числа - опционально
            # поэтому строим для этого общий пайплан, так как модели в требовании не умеют работать
            # с текстовыми данными - опционально
            ('description', Pipeline([
                ('preprocessor', FunctionTransformer(YourClassText().transform)),
                # любой векторизатор, например TF-IDF
                ('tfidf', TfidfVectorizer(....))]), 'description'),
            # любой векторизатор, например TF-IDF - опционально
            ('title_tfidf', TfidfVectorizer(), 'title'),
        ],
        remainder=...
    )),
    # Ваш классификатор, например LogReg
    ('model', LogisticRegression(....))
])

In [ ]:
X = train.drop(columns=['is_bad'])
y = train['is_bad']

# показан один из способов разбиения данных, вы можете разбить так, как считаете нужным
# либо даже обучать на всей выборке
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=SPLIT,
                                                    random_state=RANDOM_STATE)

In [ ]:
full_pipeline.fit(X_train, y_train)

## Результаты

Посмотрим на значение метрик в каждой из категории, а также на среднее значение `ROC AUC`

In [ ]:
y_pred = pd.Series(full_pipeline.predict_proba(X_test)[:, 1],
                   index=y_test.index)
calegories = np.unique(X_test.category.tolist())

roc_auc_category = {}

for cat in calegories:
    idx = X_test[X_test.category == cat].index
    roc_auc = roc_auc_score(y_test[idx], y_pred[idx])
    roc_auc_category[cat] = roc_auc
    print(f'{cat} - {roc_auc:0.2f}')

print(f'\nROC_AUC = {np.mean(list(roc_auc_category.values())):.2f}')

## Сохранение модели

In [ ]:
from joblib import dump, load

In [ ]:
dump(full_pipeline, 'my_solve.joblib') 